In [1]:
import pickle as pkl
import networkx as nx
import torch

In [74]:
with open('/usrvol/processed_tensors/SNLI/dev/sintactic/bert-base-uncased/raw/sintactic0.pkl', 'rb') as f:
    tensors = pkl.load(f)

In [38]:
tensors[0][0].label

[['Two',
  'women',
  'are',
  'embracing',
  'while',
  'holding',
  'to',
  'go',
  'packages',
  '.']]

In [39]:
tensors[0][0].edge_index

tensor([[1, 3, 3, 3, 3, 5, 5, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 5, 9, 4, 8, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [12]:
with open('/usrvol/processed_data/SNLI/dev/sintactic/sintactic0.pkl', 'rb') as f:
    data = pkl.load(f)

In [18]:
data[0][0].nodes(data=True)

NodeDataView({1: {'label': 'Two'}, 2: {'label': 'women'}, 3: {'label': 'are'}, 4: {'label': 'embracing'}, 5: {'label': 'while'}, 6: {'label': 'holding'}, 7: {'label': 'to'}, 8: {'label': 'go'}, 9: {'label': 'packages'}, 10: {'label': '.'}})

In [19]:
data[0][0].edges(data=True)

OutEdgeDataView([(2, 1, {'label': 'num'}), (4, 2, {'label': 'nsubj'}), (4, 3, {'label': 'aux'}), (4, 6, {'label': 'advcl'}), (4, 10, {'label': 'punct'}), (6, 5, {'label': 'mark'}), (6, 9, {'label': 'dobj'}), (8, 7, {'label': 'dep'}), (9, 8, {'label': 'amod'})])

In [22]:
with open('/usrvol/utils/bert-base-uncased_specific_tensors.pkl', 'rb') as f:
    tensors = pkl.load(f)

In [24]:
len(tensors)

96

In [26]:
tensors['num']

torch.Size([768])

In [41]:
data[0][0].edges(data=True)

OutEdgeDataView([(2, 1, {'label': 'num'}), (4, 2, {'label': 'nsubj'}), (4, 3, {'label': 'aux'}), (4, 6, {'label': 'advcl'}), (4, 10, {'label': 'punct'}), (6, 5, {'label': 'mark'}), (6, 9, {'label': 'dobj'}), (8, 7, {'label': 'dep'}), (9, 8, {'label': 'amod'})])

In [43]:
data[0][0].nodes(data=True)

NodeDataView({1: {'label': 'Two'}, 2: {'label': 'women'}, 3: {'label': 'are'}, 4: {'label': 'embracing'}, 5: {'label': 'while'}, 6: {'label': 'holding'}, 7: {'label': 'to'}, 8: {'label': 'go'}, 9: {'label': 'packages'}, 10: {'label': '.'}})

In [93]:
tensors[0][0].edge_label[0]

['num', 'nsubj', 'aux', 'advcl', 'punct', 'mark', 'dobj', 'dep', 'amod']

In [95]:
tensors[0][0].edge_index.size()

torch.Size([2, 19])

In [101]:
tensors[0][0].label[0]

['Two',
 'women',
 'are',
 'embracing',
 'while',
 'holding',
 'to',
 'go',
 'packages',
 '.']

In [89]:
a= tensors[0][0].edge_index[0][0:len(tensors[0][0].edge_label[0])]

In [90]:
b= tensors[0][0].edge_index[1][0:len(tensors[0][0].edge_label[0])]

In [91]:
stacked = torch.stack([torch.tensor(a), torch.tensor(b)], dim=0)

/tmp/ipykernel_26353/3442007704.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  stacked = torch.stack([torch.tensor(a), torch.tensor(b)], dim=0)


In [96]:
stacked.size()

torch.Size([2, 9])

In [106]:
triples = []
entities = []
for i in range(len(stacked[0])):
    first, second = stacked[0][i].item(), stacked[1][i].item()
    first_string, second_string, relation = tensors[0][0].label[0][first], tensors[0][0].label[0][second], tensors[0][0].edge_label[0][i]
    if not first_string in entities:
        entities.append(first_string)
    if not second_string in entities:
        entities.append(second_string)
    triples.append((first_string, relation, second_string))

In [107]:
entities

['women',
 'Two',
 'embracing',
 'are',
 'holding',
 '.',
 'while',
 'packages',
 'go',
 'to']

# Metadata

Here we generate the metadata that will receive HANConv and HGTConv initially, indicating the name of the different nodes and relations in all the data.

In [40]:
import os

In [108]:
def get_triples(pytorch_data, entities, triples):
    a = pytorch_data.edge_index[0][0:len(pytorch_data.edge_label[0])]
    b = pytorch_data.edge_index[1][0:len(pytorch_data.edge_label[0])]
    stacked = torch.stack([torch.tensor(a), torch.tensor(b)], dim=0)
    for i in range(len(stacked[0])):
                first, second = stacked[0][i].item(), stacked[1][i].item()
                first_string, second_string, relation = first.label[0][first], first.label[0][second], first.edge_label[0][i]
                if not first_string in entities:
                    entities.append(first_string)
                if not second_string in entities:
                    entities.append(second_string)
                triples.append((first_string, relation, second_string))

    return list(set(entities)), list(set(triples))

In [ ]:
entities = []
triples = []
for dataset in ['train', 'dev,', 'test']:
    folder = os.listdir(f"/usrvol/processed_tensors/SNLI/{dataset}/sintactic/bert-base-uncased/raw/")
    for file in folder:
        with open(f"/usrvol/processed_tensors/SNLI/{dataset}/sintactic/bert-base-uncased/raw/{file}", 'rb') as f:
            tensors = pkl.load(f)
        for tensor in tensors:
            first = tensor[0]
            entities, triples = get_triples(first, entities, triples)
            second = tensor[1]
            entities, triples = get_triples(second, entities, triples)

metadata = [entities, triples]

In [116]:
tensors[0][0].x[10].size()

IndexError: index 10 is out of bounds for dimension 0 with size 10

# Dataloader HGTConv, HANConv

In [ ]:
def get_triples(pytorch_data, entities, triples):
    a = pytorch_data.edge_index[0][0:len(pytorch_data.edge_label[0])]
    b = pytorch_data.edge_index[1][0:len(pytorch_data.edge_label[0])]
    stacked = torch.stack([torch.tensor(a), torch.tensor(b)], dim=0)
    for i in range(len(stacked[0])):
                first, second = stacked[0][i].item(), stacked[1][i].item()
                first_string, second_string, relation = first.label[0][first], first.label[0][second], first.edge_label[0][i]
                if not first_string in entities:
                    entities.append(first_string)
                if not second_string in entities:
                    entities.append(second_string)
                triples.append((first_string, relation, second_string))

    return list(set(entities)), list(set(triples))

In [ ]:
def get_dicts(pytorch_data, edge_tensors):
    x_dict = {}
    triples_dict = {}
    a = pytorch_data.edge_index[0][0:len(pytorch_data.edge_label[0])]
    b = pytorch_data.edge_index[1][0:len(pytorch_data.edge_label[0])]
    stacked = torch.stack([torch.tensor(a), torch.tensor(b)], dim=0)
    for i in range(len(stacked[0])):
                first, second = stacked[0][i].item(), stacked[1][i].item()
                first_string, second_string, relation = first.label[0][first], first.label[0][second], first.edge_label[0][i]
                if not first_string in entities:
                    entities.append(first_string)
                if not second_string in entities:
                    entities.append(second_string)
                specific_tensor = edge_tensors[relation]
                if not first_string in x_dict:
                    position = pytorch_data.edge_index[0][i]
                    x_dict[first_string] = pytorch_data.x[position]
                
                if not second_string in x_dict:
                    position = pytorch_data.edge_index[1][i]
                    x_dict[second_string] = pytorch_data.x[position]
                triples_dict[first_string, relation, second_string] = specific_tensor
    
    return x_dict, triples_dict
                


# Dataloader HEATConv

In [9]:
with open('/usrvol/processed_tensors/SNLI/dev/sintactic+semantic+constituency/bert-base-uncased/raw/sintactic_semantic_constituency0.pkl', 'rb') as f:
    tensors = pkl.load(f)

In [26]:
tensors[0][0].label[0]

['Two',
 'women',
 'are',
 'women',
 'while',
 'holding',
 'to',
 'go',
 'packages',
 'are',
 '«SENTENCE»',
 '«NOUN PHRASE»',
 'Two',
 '«VERB PHRASE»',
 '«VERB PHRASE»',
 'embracing',
 '«SUBORDINATE CLAUSE»',
 'while',
 '«SENTENCE»',
 '«VERB PHRASE»',
 'holding',
 '«NOUN PHRASE»',
 '«ADJECTIVE PHRASE»',
 'to',
 'go',
 'packages',
 '.']

In [27]:
node_type = []
for i in range(len(tensors[0][0].label[0])):
    if tensors[0][0].label[0][i].startswith('«') and tensors[0][0].label[0][i].endswith('»'):
        node_type.append(0)
    else:
        node_type.append(1)

In [31]:
node_type = torch.tensor(node_type, dtype=torch.long)

/tmp/ipykernel_2596/17800947.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_type = torch.tensor(node_type, dtype=torch.long)


In [45]:
from dicts import *

sintactics = list(SINTACTIC_DICT.keys())
semantics = list(SEMANTIC_DICT.keys())

edge_types = []
for i in range(len(tensors[0][0].edge_label[0])):
    if tensors[0][0].edge_label[0][i] == 'constituency relation':
        edge_types.append(2)
    elif tensors[0][0].edge_label[0][i] in sintactics:
        edge_types.append(0)
    elif tensors[0][0].edge_label[0][i] in semantics:
        edge_types.append(1)
    else:
        raise ValueError('Edge type not found')

In [48]:
edge_types = torch.tensor(edge_types, dtype=torch.long)